In [ ]:
import os
import time

import numpy as np

from depth_first_search_solver import DepthFirstSearchSolver
from parse_input import parse_itc2007_post_enrolment, parse_itc2007_curriculum_based, parse_xml
from solution_search import SolutionSearch
from util import random_gene, get_gene_maximums


In [ ]:
itc2007_track2_path = '.\\Datasets\\post'
itc2007_track3_path = '.\\Datasets\\curriculum'
itc2019_path = '.\\Datasets\\2019'

def get_all_files(path):
    return [os.path.join(dirpath, file) for dirpath, _, filenames in os.walk(path) for file in filenames]   

itc2007_track2_files = get_all_files(itc2007_track2_path)
itc2007_track3_files = get_all_files(itc2007_track3_path)
itc2019_files = get_all_files(itc2019_path)

def parse_problem (dataset,instance_path):
    if dataset == 'itc2007_track2':
        return parse_itc2007_post_enrolment(instance_path)
    elif dataset == 'itc2007_track3':
        return parse_itc2007_curriculum_based(instance_path)
    elif dataset == 'itc2019':
        return parse_xml(instance_path)

In [ ]:
def generate_random_population(dataset, instance_path, population_size, save_path):
    problem = parse_problem(dataset, instance_path)[0]
    genes = [random_gene(get_gene_maximums(problem.classes)) for _ in range(population_size)]
    
    np.save(save_path, genes)
    

In [ ]:
def generate_valid_population(dataset, instance_path, population_size, save_path,time_path):
    problem = parse_problem(dataset, instance_path)[0]
    genes = []
    times = []
    
    max_attempts_before_first_success = 1
    max_attempts = 200
    failed_attempts = 0
    
    while len(genes) < population_size and failed_attempts < max_attempts:
        
        if len(genes) == 0 and failed_attempts == max_attempts_before_first_success:
            break
        
        start_time = time.time()
        search = SolutionSearch(problem)
        solver = DepthFirstSearchSolver(search)
        solution = solver.solve(max_backtracks=2000)
        end_time = time.time()
        
        if solution['success']:
            genes.append(search.get_result_as_gene())
        else:
            failed_attempts += 1
            
        times.append((end_time - start_time,solution['success']))
                
    
    if len(genes) < population_size:
        print(f"Failed to generate valid population for {instance_path}")
    else:
        np.save(save_path, genes)
        np.savetxt(time_path, times,delimiter=',')

In [ ]:
save_path_random = f'output\\initial populations_random_{time.strftime("%Y%m%d-%H%M%S")}'
size = 100

os.mkdir(save_path_random)
os.mkdir(os.path.join(save_path_random, 'itc2007 track 2'))
os.mkdir(os.path.join(save_path_random, 'itc2007 track 3'))
os.mkdir(os.path.join(save_path_random, 'itc2019'))

for file in itc2007_track2_files:    
    generate_random_population('itc2007_track2', file, size, os.path.join(save_path_random, 'itc2007 track 2', file.split('\\')[-1].split('.')[0] + ".npy"))
    
for file in itc2007_track3_files:    
    generate_random_population('itc2007_track3', file, size, os.path.join(save_path_random, 'itc2007 track 3', file.split('\\')[-1].split('.')[0] + ".npy"))
    
for file in itc2019_files:
    generate_random_population('itc2019', file, size, os.path.join(save_path_random, 'itc2019', file.split('\\')[-1].split('.')[0] + ".npy"))
    

    
print('done')
print('save_path_random',save_path_random)

In [ ]:
save_path_valid = f'output\\initial populations_valid_{time.strftime("%Y%m%d-%H%M%S")}'

os.mkdir(save_path_valid)
os.mkdir(os.path.join(save_path_valid, 'itc2007 track 2'))
os.mkdir(os.path.join(save_path_valid, 'itc2007 track 3'))
os.mkdir(os.path.join(save_path_valid, 'itc2019'))

for file in itc2007_track2_files:    
    generate_valid_population('itc2007_track2', file, size, os.path.join(save_path_valid, 'itc2007 track 2', file.split('\\')[-1].split('.')[0] + ".npy"),os.path.join(save_path_valid, 'itc2007 track 2', file.split('\\')[-1].split('.')[0] + " time.txt"))
    
for file in itc2007_track3_files:
    generate_valid_population('itc2007_track3', file, size, os.path.join(save_path_valid, 'itc2007 track 3', file.split('\\')[-1].split('.')[0] + ".npy"),os.path.join(save_path_valid, 'itc2007 track 3', file.split('\\')[-1].split('.')[0] + " time.txt"))
    
for file in itc2019_files:
    generate_valid_population('itc2019', file, size, os.path.join(save_path_valid, 'itc2019', file.split('\\')[-1].split('.')[0] + ".npy"),os.path.join(save_path_valid, 'itc2019', file.split('\\')[-1].split('.')[0] + " time.txt"))

print('done')    
print('save_path_valid',save_path_valid)
